In [ ]:
from __future__ import print_function
from __future__ import division

import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, utils, datasets
from skimage import io, transform
from torch.autograd import Variable

import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
import pandas as pd
import random
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
torch.cuda.current_device()

In [3]:
torch.cuda.get_device_name(0)

'Quadro P1000'

In [4]:
torch.cuda.is_available()

True

In [5]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "densenet"

# Number of classes in the dataset
num_classes = 5

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 3

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Initializing DenseNet model architecture

In [7]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 320

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        model_ft.features[0] = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        input_size = 320

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

## Loading Validation And Test Data

## Test Data (202 samples) is from the human expert labeled X-rays
## Validation data is from the NLP labelled X-rays (similar to training X-rays)

In [8]:
#df = pd.read_csv('frontal_images_distance_score5.csv').iloc[:,1:]
df = pd.read_csv('frontal_images_distance_score5.csv').iloc[:,1:].sample(10000, random_state=42)
df.replace(-1,0, inplace=True)
df = df[df['DiscardFlag'] == 0]
df.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices', 'patient', 'study', 'view', 'Count', 'UniqID',
       'NewPath', 'DistanceRotatedCropped', 'DistanceOrig',
       'SSIMRotatedCropped', 'StackValue', 'DiscardFlag', 'NoFinding_Updated'],
      dtype='object')

In [9]:
df = df[['NewPath', 'Cardiomegaly', 'Edema', 'Atelectasis', 
         'Pleural Effusion', 'Consolidation']]
df.tail()

,NewPath,Cardiomegaly,Edema,Atelectasis,Pleural Effusion,Consolidation
65935,CheXpert_Processed_TrainingData/Train_70000/10...,0.0,0.0,0.0,0.0,0.0
119307,CheXpert_Processed_TrainingData/Train_120000/2...,0.0,0.0,0.0,0.0,1.0
72236,CheXpert_Processed_TrainingData/Train_75000/23...,1.0,0.0,0.0,0.0,1.0
189179,CheXpert_Processed_TrainingData/Train_190000/2...,0.0,0.0,1.0,1.0,0.0
151390,CheXpert_Processed_TrainingData/Train_155000/1...,0.0,0.0,1.0,0.0,0.0


In [10]:
dir_ = 'CheXpert-v1.0-small'
df_valid = pd.read_csv(dir_+'/'+'valid.csv')
a = df_valid['Path'].str.split("/",expand=True)
df_valid['patient']=a[2].str.replace('patient','').astype(int)
df_valid['study'] = a[3].str.replace('study','').astype(int)
df_valid['view'] = a[4].str.split('_', expand=True)[0].str.replace('view','').astype(int)
df_valid = df_valid[df_valid['Frontal/Lateral'] == 'Frontal']
df_valid['NewPath'] = df_valid['Path']
df_valid = df_valid[['NewPath', 'Cardiomegaly', 'Edema', 'Atelectasis', 
         'Pleural Effusion', 'Consolidation']]
df_valid.tail()

,NewPath,Cardiomegaly,Edema,Atelectasis,Pleural Effusion,Consolidation
229,CheXpert-v1.0-small/valid/patient64736/study1/...,0.0,0.0,0.0,0.0,0.0
230,CheXpert-v1.0-small/valid/patient64737/study1/...,0.0,0.0,0.0,0.0,0.0
231,CheXpert-v1.0-small/valid/patient64738/study1/...,1.0,1.0,0.0,0.0,0.0
232,CheXpert-v1.0-small/valid/patient64739/study1/...,0.0,0.0,0.0,0.0,0.0
233,CheXpert-v1.0-small/valid/patient64740/study1/...,0.0,0.0,1.0,1.0,0.0


In [11]:
labels_list = ['Cardiomegaly', 'Edema', 'Atelectasis', 'Pleural Effusion', 'Consolidation']
print(labels_list)

['Cardiomegaly', 'Edema', 'Atelectasis', 'Pleural Effusion', 'Consolidation']


In [12]:
train, valid = train_test_split(df, test_size=0.2, random_state=42)

## Percentage label distribution in training set

In [13]:
count_labels_df = train.iloc[:,1:9].apply(lambda x: 100 * x.value_counts()/ x.count())
count_labels_df

,Cardiomegaly,Edema,Atelectasis,Pleural Effusion,Consolidation
0.0,87.650602,75.037651,84.626004,60.504518,93.260542
1.0,12.349398,24.962349,15.373996,39.495482,6.739458


## Percentage label distribution in validation set

In [14]:
count_labels_df_valid = valid.iloc[:,1:9].apply(lambda x: 100 * x.value_counts()/ x.count())
count_labels_df_valid

,Cardiomegaly,Edema,Atelectasis,Pleural Effusion,Consolidation
0.0,89.312594,73.256397,84.947316,58.755645,92.373307
1.0,10.687406,26.743603,15.052684,41.244355,7.626693


## Calculate weight ratio of label distributions (used in loss function)

In [15]:
ratio = count_labels_df.iloc[0]/count_labels_df.iloc[1]

weights = torch.from_numpy(ratio.values).float()
weights

tensor([ 7.0976,  3.0060,  5.5045,  1.5319, 13.8380])

## PyTorch Data Loading In Mini-Batches

In [16]:
class XrayDataset(Dataset):
    
    def __init__(self, df, transform=None):
        self.xray_frame = df
        self.transform = transform
        self.y_train = np.array(self.xray_frame.iloc[:,1:9]).astype(float)
        
    def __len__(self):
        return len(self.xray_frame)

    def __getitem__(self, idx):
        img_name = self.xray_frame['NewPath'].iloc[idx]
        image = Image.open(img_name)
        target_labels = torch.from_numpy(self.y_train[idx])

        if self.transform:
            image = self.transform(image)
        sample = image, target_labels
        return sample

In [17]:
class XrayDataset_test(Dataset):
    
    def __init__(self, df, transform=None):
        self.xray_frame = df
        self.transform = transform
        self.y_train = np.array(self.xray_frame.iloc[:,1:9]).astype(float)
        
    def __len__(self):
        return len(self.xray_frame)

    def __getitem__(self, idx):
        img_name = self.xray_frame['NewPath'].iloc[idx]
        image = Image.open(img_name)
        target_labels = torch.from_numpy(self.y_train[idx])

        if self.transform:
            image = self.transform(image)
        sample = image, target_labels
        return sample

In [18]:
transform_train = transforms.Compose([
    transforms.Resize(320),
    transforms.ToTensor(),
    transforms.Normalize([0.5247], [0.2769])
    ]) 

transform_valid = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(),
    transforms.Normalize([0.5247], [0.2769])
    ]) 

In [19]:
data = {}
data['train'] = XrayDataset(train, transform=transform_train)
data['valid'] = XrayDataset_test(valid, transform=transform_train)
data['test'] = XrayDataset_test(df_valid, transform=transform_valid)

In [20]:
dataloaders_dict = {x: torch.utils.data.DataLoader(data[x], batch_size=batch_size, shuffle=False, num_workers=4) for x in ['train', 'valid', 'test']}

## User Defined Functions for Evaluating Multiple Models

In [21]:
def load_models(model):
    torch.cuda.empty_cache()
    model_ft.load_state_dict(torch.load(model, map_location=torch.device('cuda')))
    return model_ft

In [22]:
def auroc_metric(phase, model_ft):
    running_corrects = 0
    device = 'cuda'
    model_ft = model_ft.to(device)
    full_labels = []
    full_preds = []
    for batch_idx, (inputs, labels) in enumerate(dataloaders_dict[phase]):
        inputs = inputs.to(device)
        labels = labels.float()
        labels = labels.to(device)
        full_labels.append(np.array(labels.cpu()))
        with torch.no_grad():
            model_ft.eval()
            outputs = model_ft(inputs)
            outputs = torch.sigmoid(outputs)
            full_preds.append(np.array(outputs.cpu()))

    full_labels = np.concatenate(full_labels)
    full_preds = np.concatenate(full_preds)
    counts = np.sum(full_labels, axis=0)
    #auc_scores = {labels_list[i] : roc_auc_score(full_labels[:,i],full_preds[:,i]) for i in range(len(labels_list))}
    auc_scores = [roc_auc_score(full_labels[:,i],full_preds[:,i]) for i in range(len(labels_list))]
    return auc_scores, counts, full_labels, full_preds

In [23]:
def find_optimal_threshold(full_labels, full_preds):
    thresh = np.arange(0,1,0.01)
    max_class_f1 = np.zeros(5)
    best_thresholds = np.zeros(5)
    for thr in thresh:
        mcm = multilabel_confusion_matrix(full_labels, (full_preds > thr).astype(float))
        tn = mcm[:, 0, 0]
        tp = mcm[:, 1, 1]
        fn = mcm[:, 1, 0]
        fp = mcm[:, 0, 1]
        recall = tp / (tp + fn) 
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp)
        threat = tp / (tp + fp + fn)
        f1 = 2 * precision * recall / (precision + recall)
        for class_num in range(mcm.shape[0]):
            if f1[class_num] > max_class_f1[class_num]:
                max_class_f1[class_num] = f1[class_num]
                best_thresholds[class_num] = thr
    return best_thresholds

In [24]:
def recall_scores(best_thresholds, full_labels, full_preds):
    mcm = multilabel_confusion_matrix(full_labels, (full_preds > best_thresholds).astype(float))
    tn = mcm[:, 0, 0]
    tp = mcm[:, 1, 1]
    fn = mcm[:, 1, 0]
    fp = mcm[:, 0, 1]
    recall = tp / (tp + fn) 
    precision = tp / (tp + fp)
    specificity = tn / (tn + fp)
    fallout = fp / (fp + tn)
    miss = fn / (fn + tp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    return recall

## Models Evaluated:

'densenet_model_t4b_small.mdl' : 5000 samples for training + validation (80-20 split) 
                                loss not weighted by class distribution
              
'densenet_model_t7_small.mdl' : 5000 samples for training + validation (80-20 split) 
                                loss weighted by class distribution       
 
'densenet_model_t8_small.mdl' : 10000 samples for training + validation (80-20 split) 
                                loss weighted by class distribution

In [34]:
models = ['densenet_model_t4b_small.mdl',
         'densenet_model_t7_small.mdl',
         'densenet_model_t8_small.mdl']

phases = ['valid', 'test']

In [35]:
model_data = {}

for phase in phases:
    model_data[phase] = {}
    for model in models:
        model_ft = load_models(model)
        auc_scores, counts, full_labels, full_preds = auroc_metric(phase, model_ft)
        if phase == 'valid':
            valid_thresholds = find_optimal_threshold(full_labels, full_preds)
            recall_score_valid_thresh = recall_scores(valid_thresholds, full_labels, full_preds)
            model_data[phase][model] = {'AUROC': auc_scores, 'ActualLabelCount' : counts, 
                                        'Optimal Validation Threshold': valid_thresholds, 
                                        'Recall Score': recall_score_valid_thresh}
        else:
            recall_score_valid_thresh = recall_scores(valid_thresholds, full_labels, full_preds)
            test_thresholds = find_optimal_threshold(full_labels, full_preds)
            recall_score_test_thresh = recall_scores(test_thresholds, full_labels, full_preds)
            model_data[phase][model] = {'AUROC' : auc_scores, 'ActualLabelCount' : counts, 
                                        'Optimal Validation Threshold': valid_thresholds, 
                                        'Recall Score Validation Thrshld' : recall_score_valid_thresh, 
                                        'Optimal Test Threshold': test_thresholds, 
                                        'Recall Score Test Thrshld' : recall_score_test_thresh}
        

/home/varun/.conda/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/varun/.conda/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/home/varun/.conda/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/home/varun/.conda/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/varun/.conda/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/varun/.conda/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.pat

## AUC ROC Metric For Validation Data For Each Model

In [36]:
auroc_dict = {}
for phase in phases:
    auroc_dict[phase] = {}
    dict1 = model_data[phase]
    for model in models:
        auroc_dict[phase][model] = {labels_list[i] : dict1[model]['AUROC'][i] for i in range(len(labels_list))}
df = pd.DataFrame.from_dict(auroc_dict['valid']).T
df['Average ROC All Classes'] = df.agg(np.mean, axis=1)
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Average ROC All Classes
densenet_model_t4b_small.mdl,0.614591,0.749058,0.654796,0.766303,0.803528,0.717656
densenet_model_t7_small.mdl,0.609581,0.723108,0.691651,0.775819,0.775364,0.715104
densenet_model_t8_small.mdl,0.585761,0.797705,0.641506,0.771769,0.806357,0.720620


## AUC ROC Metric For Test Data For Each Model

In [37]:
df = pd.DataFrame.from_dict(auroc_dict['test']).T
df['Average ROC All Classes'] = df.agg(np.mean, axis=1)
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Average ROC All Classes
densenet_model_t4b_small.mdl,0.676220,0.760695,0.846507,0.786310,0.848392,0.783625
densenet_model_t7_small.mdl,0.688084,0.668672,0.853493,0.812946,0.827106,0.770060
densenet_model_t8_small.mdl,0.762520,0.820299,0.877941,0.817113,0.846354,0.824845


## Optimal Thresholds Calculated On Validation Data (0.0 - 1.0)

In [38]:
val_thresh_dict = {}
for phase in phases:
    val_thresh_dict[phase] = {}
    dict1 = model_data[phase]
    for model in models:
        val_thresh_dict[phase][model] = {labels_list[i] : dict1[model]['Optimal Validation Threshold'][i] for i in range(len(labels_list))}
df = pd.DataFrame.from_dict(val_thresh_dict['valid']).T
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
densenet_model_t4b_small.mdl,0.14,0.23,0.10,0.25,0.29
densenet_model_t7_small.mdl,0.47,0.70,0.61,0.52,0.47
densenet_model_t8_small.mdl,0.44,0.67,0.54,0.57,0.48


## Validation data recall scores calculated using optimal validation data thresholds

In [39]:
val_recall_dict = {}
for phase in ['valid']:
    val_recall_dict[phase] = {}
    dict1 = model_data[phase]
    for model in models:
        val_recall_dict[phase][model] = {labels_list[i] : dict1[model]['Recall Score'][i] for i in range(len(labels_list))}
df = pd.DataFrame.from_dict(val_recall_dict['valid']).T
df['Average Recall All Classes'] = df.agg(np.mean, axis=1)
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Average Recall All Classes
densenet_model_t4b_small.mdl,0.750000,0.394366,0.605263,0.666041,0.823601,0.647854
densenet_model_t7_small.mdl,0.740000,0.469484,0.730263,0.784240,0.840633,0.712924
densenet_model_t8_small.mdl,0.643333,0.539906,0.730263,0.778612,0.828467,0.704116


## Test data recall scores calculated using optimal validation data thresholds

In [40]:
test_recall_dict = {}
for phase in ['test']:
    test_recall_dict[phase] = {}
    dict1 = model_data[phase]
    for model in models:
        test_recall_dict[phase][model] = {labels_list[i] : dict1[model]['Recall Score Validation Thrshld'][i] for i in range(len(labels_list))}
df = pd.DataFrame.from_dict(test_recall_dict['test']).T
df['Average Recall All Classes'] = df.agg(np.mean, axis=1)
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Average Recall All Classes
densenet_model_t4b_small.mdl,0.000000,0.000000,0.0000,0.000000,0.515625,0.103125
densenet_model_t7_small.mdl,0.826667,0.272727,0.9375,0.642857,0.718750,0.679700
densenet_model_t8_small.mdl,0.666667,0.166667,0.8750,0.809524,0.718750,0.647321


## Optimal Thresholds Calculated On Test Data (0.0 - 1.0)

In [41]:
val_thresh_dict = {}
for phase in ['test']:
    val_thresh_dict[phase] = {}
    dict1 = model_data[phase]
    for model in models:
        val_thresh_dict[phase][model] = {labels_list[i] : dict1[model]['Optimal Test Threshold'][i] for i in range(len(labels_list))}
df = pd.DataFrame.from_dict(val_thresh_dict['test']).T
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
densenet_model_t4b_small.mdl,0.10,0.07,0.10,0.27,0.33
densenet_model_t7_small.mdl,0.42,0.38,0.59,0.58,0.43
densenet_model_t8_small.mdl,0.38,0.23,0.63,0.58,0.51


## Test data recall scores calculated using optimal test data thresholds

In [42]:
test_recall_dict = {}
for phase in ['test']:
    test_recall_dict[phase] = {}
    dict1 = model_data[phase]
    for model in models:
        test_recall_dict[phase][model] = {labels_list[i] : dict1[model]['Recall Score Test Thrshld'][i] for i in range(len(labels_list))}
df = pd.DataFrame.from_dict(test_recall_dict['test']).T
df['Average Recall All Classes'] = df.agg(np.mean, axis=1)
df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Average Recall All Classes
densenet_model_t4b_small.mdl,0.920000,0.878788,0.68750,0.619048,0.734375,0.767942
densenet_model_t7_small.mdl,0.880000,0.757576,0.84375,0.642857,0.843750,0.793587
densenet_model_t8_small.mdl,0.826667,0.878788,0.84375,0.785714,0.687500,0.804484
